In [1]:
import pandas as pd

In [2]:
excel_dbfs_path = f"./david_finance.xlsx"
sheet_name = f"cargos_pue"

In [3]:
dfp = pd.read_excel(excel_dbfs_path, engine = "openpyxl", sheet_name = sheet_name, dtype = {"monto":'float'})

In [4]:
tdc_df = pd.read_excel(excel_dbfs_path, engine = "openpyxl", sheet_name = "tarjetas_de_credito")

In [5]:
class Transactions:
    def __init__(self, df: pd.DataFrame = None):
        self.empty_recurrents_df = pd.DataFrame(
                        columns=['fecha_de_cargo' ,
                                'comercio',
                                'descripcion', 
                                'categoria', 
                                'monto', 
                                'forma_de_pago', 
                                'realizado_por', 
                                'limite_de_pago', 
                                'deuda_a_plazos_recurrente_o_normal', 
                                'comentarios'
                                ])

        self.tdc_df = pd.read_excel(excel_dbfs_path, engine = "openpyxl", sheet_name = "tarjetas_de_credito")
        self.condition_blank_comment = lambda text: text if text == text else ""
        self.df = df
        self.validate_if_empty()

    def validate_if_empty(self):
        if len(self.df) == 0:
            raise ImportError(f"Unable to load dataframe of length {len(self.df)}. Load at least 1 len dataframe")
        
        
    def get_due_dates(self):
        """
        :ref:`row` is the current iteration row
        """
        # To get the due date we need:
        # - to merge with credit cards DimTable to get the closing date and the grace period
        df_dias_de_corte = self.df.merge(
            right = self.tdc_df,
            how="left",
            left_on = "forma_de_pago",
            right_on = "credit_cards"
        ) 

        
        # - to substitue the actual transaction date with the day closing date. Just apply for credit cards
        df_dias_de_corte["limite_de_pago"] = pd.to_datetime(
                [
                    f'{row["fecha_de_cargo"].year}-{row["fecha_de_cargo"].month}-{int(row["dia_de_corte"])}' 
                    if row["credit_cards"] == row["credit_cards"] # means if not null
                    else row["fecha_de_cargo"] 
                    for _,row in df_dias_de_corte.iterrows()
                ], format = "%Y-%m-%d"
                )
        # - Impute with 0 no credit cards transactions
        df_dias_de_corte["plazo_para_pagar"] = df_dias_de_corte["plazo_para_pagar"].fillna(0).copy()

        # - Get delta days based on the grace period per credit card (0 for no credit cards transactions)
        days_to_add = pd.to_timedelta(df_dias_de_corte["plazo_para_pagar"], unit='D') 

        # - Sum the grace period 
        df_dias_de_corte["limite_de_pago"] += days_to_add
        
        # - Sum one month if closing date was after the transaction date
        df_dias_de_corte["limite_de_pago"] = (df_dias_de_corte["limite_de_pago"] + pd.DateOffset(months=1)).where(
            df_dias_de_corte['fecha_de_cargo'].dt.day > df_dias_de_corte['dia_de_corte'],
            df_dias_de_corte["limite_de_pago"])

        df_dias_de_corte.drop(columns=["credit_cards","dia_de_corte", "plazo_para_pagar"], inplace=True)
        df_dias_de_corte["deuda_a_plazos_recurrente_o_normal"] = "Normal"
        df_dias_de_corte["comentarios"] = df_dias_de_corte["comentarios"].apply(self.condition_blank_comment)

        return df_dias_de_corte
    
    def pdconcat_wise(self, base_df: pd.DataFrame, concat_df: pd.DataFrame):
        if concat_df is None:
            return base_df
        elif len(base_df) > 0:
            return pd.concat(
                        [
                            base_df,
                            concat_df
                        ],
                            ignore_index=True
                    )
        else:
            return concat_df

In [13]:
transactions_creator = Transactions(dfp)

In [14]:
transactions_df = transactions_creator.get_due_dates()

In [15]:
transactions_df

,fecha_de_cargo,comercio,descripcion,categoria,monto,forma_de_pago,realizado_por,comentarios,limite_de_pago,deuda_a_plazos_recurrente_o_normal
0,2024-09-30,Costco,Costco,Despensa,2585.35,Costco,David,Dia de madres,2024-11-04,Normal
1,2024-09-30,Heb,Heb despensa,Despensa,604.39,Costco,NaN,,2024-11-04,Normal


In [16]:
%store -r plazos_df
%store -r recurrentes_df

In [17]:
all_spend_df = transactions_creator.pdconcat_wise(transactions_df, plazos_df)
all_spend_df = transactions_creator.pdconcat_wise(all_spend_df, recurrentes_df)

In [18]:
%store all_spend_df

Stored 'all_spend_df' (DataFrame)
